In [1]:
from src import updated_cnc_tableau as cnc
import numpy as np
import timeit
from random import choice

# Boolean map using GHZ state

A GHZ state $\left | \text{GHZ}\right\rangle = \left(\left | 000\right\rangle+\left | 111\right\rangle\right)/\sqrt{2}$ can be described by a stabilizer group $S_{\text{GHZ}}$ whose generators can be written as:
$$ X_1X_2\cdots X_n\quad \text{and}\quad -Y_1X_2\cdots X_{i-1}Y_iX_{i+1}\cdots X_n,~~(i=1,\cdots,n). $$

We can measure the eigenvalues of these generators by performing a sequence of local Pauli $X_i$, $Y_i$ ($i=1,\cdots,n$) measurements on the GHZ state according to which Pauli matrix appears on qubit $i$. For example, to measure the eigenvalue of the $X_1\cdots X_n$ Pauli operator we measure $X_i$ for all $i=1,\cdots,n$. The outcomes will be locally random, but their $\text{mod} 2$ sum will be deterministic.

This protocol can be thought of as computing a Boolean map on a subset of $\mathbb{Z}_{2}^{2n}$. In particular, consider $D\subset \mathbb{Z}_{2}^{2n}$ which consists of the identity $\bar 0 = (0,\cdots,0)$, as well as for all $i=1,\cdots,n$ bit strings of the form: $a^{(i)}\in \mathbb{Z}_{2}^{2n}$ such that $a^{(i)}_1 = a^{(i)}_i = 1$, and $a^{(i)}_j=0$, otherwise. Then we have that $f_{\text{GHZ}}:D\to \mathbb{Z}_2$ such that $f_{\text{GHZ}}(\bar 0) = 0$ and $f_{\text{GHZ}}(a^{(i)}) = 1$.

Each local Pauli measurement $X_i,Y_j$ anticommutes with at least one generators of $S_{\text{GHZ}}$ and thus our tableau simulator will perform a Case II update. The correctness of the updates can be checked indirectly by comparing with the outputs of the known Boolean map $f_{\text{GHZ}}$.

In [10]:
# by default our initial state is |+>\otimes n:
def ghz_state_prep(simulator,n):
    for m in range(1,n):
        simulator.apply_hadamard(m)
    for m in range(1,n):
        simulator.apply_cnot(0,m)

In [37]:
# Order the measurement bases as: 0 -> X1...XN, then i -> Y1...Xi-1 Yi Xi+1 ... Xn:
def measurement_bases(n):
    # Initialize empty 3D array
    bases_array = np.empty((0, n, 2*n), dtype=int)

    # Create the first measurement basis
    zero_matrix = np.zeros((n, n), dtype=int)
    identity_matrix = np.eye(n, dtype=int)

    # Stack and expand the dimension to make it 3D
    first_basis = np.hstack((identity_matrix, zero_matrix))[np.newaxis, :, :]

    # initialize empty row vector:
    yint = np.zeros((1,2*n),dtype=int)

    # modify first row:
    y1 = yint.copy()
    y1[0,n] = 1
    y1[0,0] = 1

    # Concatenate along the first axis
    bases_array = np.concatenate((bases_array, first_basis), axis=0)

    # generate remaining measurements:
    for k in range(1,n):
        # initialize measurement bases:
        basis = first_basis.copy()
        basis[0,0,:] = y1

        # modify kth row:
        yk = yint.copy()
        yk[0,k] = 1
        yk[0,k+n] = 1
        basis[0,k,:] = yk

        # Concatenate along the first axis
        bases_array = np.concatenate((bases_array, basis), axis=0)
        

    return bases_array

In [72]:
def ghz_function(n):
    boolean_mapping = dict()
    for k in range(n):
        if k == 0:
            image = 0
        else:
            image = 1
        
        boolean_mapping[k] = image
    return boolean_mapping

In [78]:
# check results for range of qubit values:
for n in range(2,50): 
    # fix n-qubits:
    m = 1

    # initialize measurements:
    measurements = measurement_bases(n)

    # initialize empty list for outcomes:
    outcomes_array = []
    for i in range(n):

        # initialize simulator:
        simulator = cnc.CncSimulator(n,m)

        # prepare ghz state
        ghz_state_prep(simulator,n)

        bases = measurements[i,:,:]
        outcomes = []
        for j in range(n):
            outcome = simulator.measure(bases[j, :])
            outcomes.append(outcome)
        outcomes_array.append(outcomes)

    # check results:
    mapping = ghz_function(n)
    xor_outcomes = [sum(outcomes_array[k]) % 2 for k in range(n)]
    bool_list = [mapping[k] == xor_outcomes[k] for k in range(n)]
    # print results
    print(f"Simulation for {n} qubits gives correct result:", all(bool_list),"\n")

Simulation for 2 qubits gives correct result: True 

Simulation for 3 qubits gives correct result: True 

Simulation for 4 qubits gives correct result: True 

Simulation for 5 qubits gives correct result: True 

Simulation for 6 qubits gives correct result: True 

Simulation for 7 qubits gives correct result: True 

Simulation for 8 qubits gives correct result: True 

Simulation for 9 qubits gives correct result: True 

Simulation for 10 qubits gives correct result: True 

Simulation for 11 qubits gives correct result: True 

Simulation for 12 qubits gives correct result: True 

Simulation for 13 qubits gives correct result: True 

Simulation for 14 qubits gives correct result: True 

Simulation for 15 qubits gives correct result: True 

Simulation for 16 qubits gives correct result: True 

Simulation for 17 qubits gives correct result: True 

Simulation for 18 qubits gives correct result: True 

Simulation for 19 qubits gives correct result: True 

Simulation for 20 qubits gives corre